In [58]:
from scenedetect import detect, AdaptiveDetector, ThresholdDetector
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle
from itertools import compress
from textwrap import wrap
class AllVids():
    def __init__(self):
        self.List={"Name":[], 'scenelist':[],'Frametime':[],'Duration':[],'Cuts':[],'AvgD':[]}
        self.MINUTE=1
        self.sec_divider=4
        N=self.MINUTE*self.sec_divider+1
        self.cut_time=np.linspace(0,self.MINUTE,N)

    def add_video(self,name,scenelist):
        self.List['Name'].append(name)
        self.List['scenelist'].append(scenelist)
        self.List['Duration'].append([])
        self.List['Frametime'].append([])
        self.List['Cuts'].append([])
        self.List['AvgD'].append([])
    def show_name(self):
        print(self.List['Name'])
        print(self.List['Frametime'])
        print(self.List['Duration'])

    def count_vids(self):
        print(len(self.List))
    
    def set_MINUTE(self,MINUTE,sec_divider):
        self.MINUTE=MINUTE
        self.sec_divider=sec_divider
        N=self.MINUTE*self.sec_divider+1
        self.cut_time=np.linspace(0,self.MINUTE,N)
        
        
    
    def addcuts(self):
        N=self.MINUTE*self.sec_divider+1 #sets up the number of bins by getting the number of minutes and the dividins
        x=self.cut_time
        for i,n in enumerate(self.List['Name']):
            FRT=self.List['Frametime'][i]
            N=self.MINUTE*self.sec_divider+1
            count=np.empty(N) * np.nan
            avgd=np.empty(N) * np.nan
            for j in range(N):
                #print('x[j]: '+str(x[j])+' x[j]+: '+ str(x[j]+1/self.sec_divider))
                #print(list(compress(self.List['Frametime'][i],(FRT > x[j]) & (FRT <= x[j]+1/self.sec_divider))))

                dummy=((FRT > x[j]) & (FRT <= x[j]+1/self.sec_divider)).sum()
                #print('Count:'+ str(dummy))
                if x[j]>FRT[-1]:
                    dummy=np.nan

                dummy2=list(compress(self.List['Duration'][i],(FRT > x[j]) & (FRT <= x[j]+1/self.sec_divider)))
                if len(dummy2)!=0:
                    avgd[j]=(sum(dummy2)/len(dummy2))
                    #print('Scene duration:'+ str(dummy2))
                    #print('Average:'+ str(sum(dummy2)/len(dummy2)))
                elif x[j]<FRT[-1]:
                    avgd[j]=avgd[j-1]
                    
                count[j]=dummy
            self.List['Cuts'][i]=[]    
            self.List['Cuts'][i]=count
            self.List['AvgD'][i]=[]    
            self.List['AvgD'][i]=avgd

    def add_duration(self):        
        for i,n in enumerate(self.List['Name']):
            scl=self.List['scenelist'][i]
            duration=[]
            FrameTime=[]
            
            for j in range(len(scl)):

                #print('From: ' + str(scene_list[i][0])+ ' to: '+ str(scene_list[i][1]))
                tim=datetime.strptime(str(scl[j][0]),'%H:%M:%S.%f') #gets the start time of the scene in question
                if (tim.minute)<self.MINUTE:
                    dur=datetime.strptime(str(scl[j][1]-scl[j][0]),'%H:%M:%S.%f')
                    duration.append(dur.second+(dur.microsecond/1000000))#finds the difference between the end time and the start time of the scene
                    FrameTime.append((tim.minute)+(tim.second/60)+(tim.microsecond/60000000))
            self.List['Duration'][i]=[]
            self.List['Frametime'][i]=[]
            self.List['Duration'][i]=duration
            self.List['Frametime'][i]=FrameTime
        self.addcuts()

class ShortVids():
    def __init__(self):
        self.List={"Name":[], 'scenelist':[],'Frametime':[],'Duration':[]}
        self.MINUTE=1
        self.sec_divider=4
        N=self.MINUTE*self.sec_divider+1
        self.cut_time=np.linspace(0,self.MINUTE,N)

    def add_video(self,name,scenelist):
        self.List['Name'].append(name)
        self.List['scenelist'].append(scenelist)
        self.List['Duration'].append([])
        self.List['Frametime'].append([])

    def show_name(self):
        print(self.List['Name'])
        print(self.List['scenelist'])
        print(self.List['scenelist'][-1])
        print(self.List['AvgD'])

    def set_MINUTE(self,MINUTE,sec_divider):
        self.MINUTE=MINUTE
        self.sec_divider=sec_divider
        N=self.MINUTE*self.sec_divider+1
        self.cut_time=np.linspace(0,self.MINUTE,N)
        
    
    def add_duration(self):        
        for i,n in enumerate(self.List['Name']):
            scl=self.List['scenelist'][i]
            duration=[]
            FrameTime=[]
            
            for j in range(len(scl)):

                #print('From: ' + str(scene_list[i][0])+ ' to: '+ str(scene_list[i][1]))
                tim=datetime.strptime(str(scl[j][0]),'%H:%M:%S.%f') #gets the start time of the scene in question
                if (tim.minute)<self.MINUTE:
                    dur=datetime.strptime(str(scl[j][1]-scl[j][0]),'%H:%M:%S.%f')#finds the difference between the end time and the start time of the scene
                    duration.append(dur.second+(dur.microsecond/1000000))
                    FrameTime.append((tim.minute)+(tim.second/60)+(tim.microsecond/60000000))
            self.List['Duration'][i]=[]
            self.List['Frametime'][i]=[]
            self.List['Duration'][i]=duration
            self.List['Frametime'][i]=FrameTime


In [ ]:
import os
# folder path
Folder_name='MarkLong' #the name of the folder with all of the .mp4 files
dir_path = 'C:/Users/...'+Folder_name #add the workspace folder

# list to store files
res = []
# Iterate directory
for file in os.listdir(dir_path):
    # check only text files
    if file.endswith('.mp4'):
        res.append(dir_path+'/'+file)

MarkVids=AllVids()

for nam in res:
    #print(nam)
    sclist=[]
    sclist=detect(nam, AdaptiveDetector())
    '''scene list outupts a list of each scene with and start time and end time formatted like this
    
    start time                         ,end time
    (00:00:00.000 [frame=0, fps=23.976], 00:00:01.168 [frame=28, fps=23.976]),
    (00:00:01.168 [frame=28, fps=23.976], 00:00:04.087 [frame=98, fps=23.976]),
    (00:00:04.087 [frame=98, fps=23.976], 00:00:09.551 [frame=229, fps=23.976]),
    ...
    
    '''
    MarkVids.add_video(nam,sclist)
MarkVids.set_MINUTE(30,2)
MarkVids.add_duration()

# Open a file and use dump() 
with open(Folder_name+'.pkl', 'wb') as Stored: 
      
    # A new file will be created 
    pickle.dump(MarkVids, Stored) 

In [ ]:

MarkVids.show_name()
MarkVids.List

In [ ]:
import os
# folder path
dir_path = 'C:/Users/....'

# list to store files
res = []
# Iterate directory
for file in os.listdir(dir_path):
    # check only text files
    if file.endswith('.mp4'):
        res.append(dir_path+'/'+file)

CleoVids=ShortVids()
for nam in res:
    sclist=[]
    sclist=detect(nam, AdaptiveDetector())
    CleoVids.add_video(nam,sclist)

CleoVids.set_MINUTE(1,90)
CleoVids.add_duration()
with open('CleoStored.pkl', 'wb') as Stored: 
    # A new file will be created 
    pickle.dump(CleoVids, Stored) 


In [7]:
import os
# folder path
dir_path = 'C:/Users/...'

# list to store files
res = []
# Iterate directory
for file in os.listdir(dir_path):
    # check only text files
    if file.endswith('.mp4'):
        res.append(dir_path+'/'+file)

MarkShorts=ShortVids()
for nam in res:
    sclist=[]
    sclist=detect(nam, AdaptiveDetector())
    MarkShorts.add_video(nam,sclist)
import pickle
# Open a file and use dump() 

MarkShorts.set_MINUTE(1,90)
MarkShorts.add_duration()
with open('MarkShorts.pkl', 'wb') as Stored: 
      
    # A new file will be created 
    pickle.dump(MarkShorts, Stored) 


In [8]:
import os
# folder path
dir_path = 'C:/Users...'

# list to store files
res = []
# Iterate directory
for file in os.listdir(dir_path):
    # check only text files
    if file.endswith('.mp4'):
        res.append(dir_path+'/'+file)

Vsauce=ShortVids()
for nam in res:
    sclist=[]
    sclist=detect(nam, AdaptiveDetector())
    Vsauce.add_video(nam,sclist)
import pickle
# Open a file and use dump() 

Vsauce.set_MINUTE(1,90)
Vsauce.add_duration()
with open('Vsauce.pkl', 'wb') as Stored: 
      
    # A new file will be created 
    pickle.dump(Vsauce, Stored) 


In [9]:
import os
# folder path
dir_path = 'C:/Users/...'

# list to store files
res = []
# Iterate directory
for file in os.listdir(dir_path):
    # check only text files
    if file.endswith('.mp4'):
        res.append(dir_path+'/'+file)

Veri=ShortVids()
for nam in res:
    sclist=[]
    sclist=detect(nam, AdaptiveDetector())
    Veri.add_video(nam,sclist)
import pickle
# Open a file and use dump() 

Veri.set_MINUTE(1,90)
Veri.add_duration()
with open('Veri.pkl', 'wb') as Stored: 
      
    # A new file will be created 
    pickle.dump(Veri, Stored) 
